In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('quora_train_sample.csv')

In [3]:
df.head()

,Unnamed: 0,qid,question_text,target
0,781152,990588f6dfebf5a8161e,"Why is Trump in trouble for the ""shithole"" rem...",1
1,147077,1cc35afd2b752c94cde6,Would I have a chance of getting an internship...,0
2,690610,8742ebb59f819cbcf308,"Do students at schools like Princeton, Columbi...",0
3,173444,21eac90fbe362155ea06,Where can I get quick notifications about UPSC...,0
4,27301,0559366f2e08fc9e8432,"How is the job security right now of BTA, join...",0


In [4]:
df['question_text'].tolist()[:5]

['Why is Trump in trouble for the "shithole" remark when it\'s only Democrats who claim he did it?',
 'Would I have a chance of getting an internship in the U.S. Senate if I have a 2.96 GPA?',
 'Do students at schools like Princeton, Columbia, Yale, Harvard and Stanford wear designer brands? How do less wealthy students react?',
 'Where can I get quick notifications about UPSC 2018?',
 'How is the job security right now of BTA, joining from TCS with 2.5 years of experience, down the line in Deloitte USI (consulting)?']

In [5]:
df.shape

(10000, 4)

In [6]:
df['target'].value_counts(normalize=True)

target
0    0.9365
1    0.0635
Name: proportion, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.25, random_state= 42)

In [8]:
train.shape, test.shape

((7500, 4), (2500, 4))

In [9]:
train.head()

,Unnamed: 0,qid,question_text,target
4901,822748,a13799d4fced124bde34,What does the bass mean in a song?,0
4375,240478,2f0a9acfbdb7045b7d86,How do I invest £50000 in the best way possible?,0
6698,68519,0d71093a7abb9e5e1fba,I’ve heard ministers make inside jokes about t...,0
9805,332284,4122f78885c053dcae07,What are the best ways to smoke a boneless tur...,0
1101,147551,1cdc4872bd2f52aa3fbd,What are some good speech topics for grade 12?,0


# Simple Approach

In [10]:
test['pred']=0

In [11]:
from sklearn.metrics import confusion_matrix, classification_report

In [12]:
def evaluation_report(gt, pred):
    print(confusion_matrix(gt, pred))
    print(classification_report(gt, pred))

print("TEST")
evaluation_report(test['target'], test['pred'])

TEST
[[2321    0]
 [ 179    0]]
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      2321
           1       0.00      0.00      0.00       179

    accuracy                           0.93      2500
   macro avg       0.46      0.50      0.48      2500
weighted avg       0.86      0.93      0.89      2500



C:\Users\arvenka\AppData\Local\anaconda3\envs\tensorflow\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\arvenka\AppData\Local\anaconda3\envs\tensorflow\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\arvenka\AppData\Local\anaconda3\envs\tensorflow\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

In [13]:
# sincere
train[train['target'] == 0].sample(n= 10)['question_text'].tolist()

['How can I know if I have internal bleeding in my foot?',
 'What was life like in Indonesia during the 80s?',
 'Why are there so many remaining vacancies in US federal government "key jobs"?',
 'Why do my legs get swollen and painful after eating wheat products?',
 'What is the difference between US highways and Interstate highways in terms of road and infrastructure quality?',
 'What do you think of Teri Hatcher as an actress?',
 'Will Ben Sasse likely seek higher officer one day?',
 'What causes coulrophobia in a culture?',
 'What qualifications are needed to be a room leader in a Preschool Childcare setting?',
 'What are important things while creating ERC20 tokens for ICO? After getting tokens in wallet. How do Android and iOS apps are created to link with this wallet?']

In [14]:
# Insincere
train[train['target'] == 1].sample(n= 10)['question_text'].tolist()

['Are anime traps gay?',
 'Why do the British get angry at Irish Americans funding the IRA while ignoring the support their security forces gave to loyalist paramilitary groups in Northern Ireland?',
 'Do Tamil Nadu girls like to have sex with strangers?',
 'On Quora, I get a growing awareness of the USA being increasingly plagued by conservative ignorance and a sense of US exceptionalism (seemingly due to highly efficient propaganda). Can it continue indefinitely, and what would it take to change that?',
 'Is the MSM, deep state actors and AstroTurf social media paid recruiters trying hard to impeach a honest President Trump?',
 'Is it my imagination or is Mitch McConnell and his cronies walking around with hard-ons now that they are "in charge"?',
 "Why doesn't the UK realise that the government wanted Brexit and played us via the media, etc. to get the result they wanted?",
 'Is it True that Muslims Refugees in Sweden rape women all the time and the Swedish government ignore there c

# Baseline approach

In [15]:
insincere_intution = ['black', 'trump', 'dumb', 'gays', 'LDS', 'girl', 'blond', 'freak', 'sex', 'bitch', 'dictator', 'lie', 'radical']

In [16]:
def baseline_classifier(txt):
    #if len(txt.split())< 2:
     #   print(txt)
      #  return 1

    for i in insincere_intution:
        if i.lower() in txt.lower():
            return 1

    return 0

In [17]:
baseline_classifier('How can america be great again if we keep on addressing president trump as "Don the con"?')

1

In [18]:
train['baseline_predictions'] = train['question_text'].apply(lambda x:baseline_classifier(x))

In [19]:
test['baseline_predictions'] = test['question_text'].apply(lambda x:baseline_classifier(x))

In [20]:
train['baseline_predictions'].value_counts()

baseline_predictions
0    7053
1     447
Name: count, dtype: int64

In [21]:
print('TRAINING')
evaluation_report(train['target'], train['baseline_predictions'])
print('TESTING')
evaluation_report(test['target'], test['baseline_predictions'])

TRAINING
[[6712  332]
 [ 341  115]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      7044
           1       0.26      0.25      0.25       456

    accuracy                           0.91      7500
   macro avg       0.60      0.60      0.60      7500
weighted avg       0.91      0.91      0.91      7500

TESTING
[[2208  113]
 [ 141   38]]
              precision    recall  f1-score   support

           0       0.94      0.95      0.95      2321
           1       0.25      0.21      0.23       179

    accuracy                           0.90      2500
   macro avg       0.60      0.58      0.59      2500
weighted avg       0.89      0.90      0.89      2500



In [22]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer


In [41]:
nltk.download('stopwords')
nltk.download('punkt_tab')
Stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arvenka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\arvenka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [42]:
Stopwords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [43]:
 white_list = ['weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
'who',
 'whom',
 'why',
 'will']

In [44]:
set(white_list)

{'weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
 'who',
 'whom',
 'why',
 'will'}

In [36]:
Stopwords = Stopwords-set(white_list)

In [52]:
Stopwords

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [53]:
import re

In [67]:
wml = WordNetLemmatizer()
def lemmatise(lowercase_word):
    lemma = []
    for word in lowercase_word:
        tokens = wml.lemmatize(word)
        lemma.append(tokens)
    return lemma

ps = PorterStemmer()
def stemming(lw):
    stems = []
    for word in lw:
        tokens = ps.stem(word)
        stems.append(tokens)
    return stems
    

In [69]:
def data_cleaning_pipeline(text, stopwords, compression_technique='stem'):
    #lowercase
    text = text.lower()

    #remove punctuations
    text = re.sub(r'[^\w\s]', '', text)

    #tokenize
    tokens = word_tokenize(text)

    #tokens
    tokens = [i for i in tokens if i not in stopwords]

    # Text standardization
    if compression_technique == 'stem':
        tokens = stemming(tokens)
    else:
        tokens = lemmatise(tokens)
        
    return tokens

In [62]:
train['question_text'].tolist()[0]

'What does the bass mean in a song?'

In [71]:
data_cleaning_pipeline(train['question_text'].tolist()[10], Stopwords)

['professor', 'tyron', 'hay', 'prove', 'homosexu', 'genet', 'disord']

In [73]:
train['question_text_cleaned'] = train['question_text'].apply(data_cleaning_pipeline, args=(Stopwords,))
test['question_text_cleaned'] = test['question_text'].apply(data_cleaning_pipeline, args=(Stopwords,))


In [74]:
train.head()

,Unnamed: 0,qid,question_text,target,baseline_predictions,question_text_cleaned
4901,822748,a13799d4fced124bde34,What does the bass mean in a song?,0,0,"[bass, mean, song]"
4375,240478,2f0a9acfbdb7045b7d86,How do I invest £50000 in the best way possible?,0,0,"[invest, 50000, best, way, possibl]"
6698,68519,0d71093a7abb9e5e1fba,I’ve heard ministers make inside jokes about t...,0,1,"[ive, heard, minist, make, insid, joke, peopl,..."
9805,332284,4122f78885c053dcae07,What are the best ways to smoke a boneless tur...,0,0,"[best, way, smoke, boneless, turkey, breast]"
1101,147551,1cdc4872bd2f52aa3fbd,What are some good speech topics for grade 12?,0,0,"[good, speech, topic, grade, 12]"
